# Daniel's Pre-Processing


In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Number of GPUs:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

CUDA available: True
CUDA version: 12.9
Number of GPUs: 1
GPU name: NVIDIA GeForce MX450


Daniel's Branch Created


In [ ]:
import random

def set_seed(seed_value=42):
    """Sets the seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
        # The two lines below can sometimes slow down training but ensure full reproducibility
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Call the function at the beginning of your script
set_seed()

In [2]:
!pip install gensim


In [3]:
import pandas as pd
import numpy as np

In [4]:
train_df = pd.read_json('../Dataset/train.json')
test_df = pd.read_json('../Dataset/test.json')
validation_df = pd.read_json('../Dataset/validation.json')

In [5]:
train_df.head()

,premise,hypothesis,label
0,Pluto rotates once on its axis every 6.39 Eart...,Earth rotates on its axis once times in one day.,neutral
1,---Glenn =====================================...,Earth rotates on its axis once times in one day.,entails
2,geysers - periodic gush of hot water at the su...,The surface of the sun is much hotter than alm...,neutral
3,Facts: Liquid water droplets can be changed in...,Evaporation is responsible for changing liquid...,entails
4,"By comparison, the earth rotates on its axis o...",Earth rotates on its axis once times in one day.,entails


In [6]:
train_df.tail()

,premise,hypothesis,label
23083,"which is not only the motion of our bodies, bu...",Work is done only if a force is exerted in the...,neutral
23084,"The Red Star, that celestial curse whose eccen...",Red-shift refers to a shift toward red in the ...,neutral
23085,The lines in the spectrum of a luminous body s...,Red-shift refers to a shift toward red in the ...,entails
23086,The radial velocity of a star away from or tow...,Red-shift refers to a shift toward red in the ...,entails
23087,This expansion causes the light from distant s...,Red-shift refers to a shift toward red in the ...,entails


In [7]:
test_df.head()

,premise,hypothesis,label
0,Based on the list provided of the uses of subs...,"If a substance has a ph value greater than 7,t...",neutral
1,If one or two base pairs are change...,Invertebrates (and higher animals) can also be...,neutral
2,"At high temperatures, the solid dye converts i...",Gases and liquids become solids at low tempera...,neutral
3,Chapter 11 Gas and Kinetic Theory .,The behavior of ideal gases is explained by ki...,neutral
4,Both the continental crust and the oceanic cru...,Gabbro is a dark dense rock that can be found ...,neutral


# Data Pre-Processing


In [8]:
import nltk
if not nltk.download('punkt'):
    nltk.download('punkt')
    nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

if not nltk.download('stopwords'):
    nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DCCN9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DCCN9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# These are just common English contractions. We used it in Lab 5 before!
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have",
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did",
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have",
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us",
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not",
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have",
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have",
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will",
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have",
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}

In [10]:
# Stemmer and Lemmatizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize

if not nltk.download('wordnet'):
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DCCN9\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
# Helper function for lemmatization with POS tagging
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no match

In [12]:
# Allowed POS tags for filtering (example: nouns, verbs, adjectives, adverbs)
allowed_pos_tags = {'NN', 'NNS', 'NNP', 'NNPS',   # Nouns
                    'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ',  # Verbs
                    'JJ', 'JJR', 'JJS',           # Adjectives
                    'RB', 'RBR', 'RBS'}           # Adverbs

In [13]:
# Filter the desired POS tags
def filter_tokens_by_pos(tokens):
    tagged_tokens = pos_tag(tokens)
    filtered = [word for word, tag in tagged_tokens if tag in allowed_pos_tags]
    return filtered

In [14]:
# Config
MIN_WORD_LENGTH = 3
MAX_WORD_LENGTH = 124 

In [15]:
def clean_dataset(dataset, MIN_WORD_LENGTH=3, MAX_WORD_LENGTH=50, method=None, pos_filter=False, stop_w=False):
    cleaned_premises = []
    cleaned_hypotheses = []
    cleaned_labels = []

    for index, row in dataset.iterrows():
        premise = row['premise']
        hypothesis = row['hypothesis']
        label = row['label']

        # Lowercase
        premise = premise.lower()
        hypothesis = hypothesis.lower()

        # Expand contractions
        for contraction, full_form in contraction_dict.items():
            premise = premise.replace(contraction, full_form)
            hypothesis = hypothesis.replace(contraction, full_form)

        # Remove punctuation/special chars
        premise = re.sub(r'[^\w\s]', '', premise)
        hypothesis = re.sub(r'[^\w\s]', '', hypothesis)

        # Normalize whitespace
        premise = re.sub(r'\s+', ' ', premise).strip()
        hypothesis = re.sub(r'\s+', ' ', hypothesis).strip()

        # Tokenization
        premise_tokens = word_tokenize(premise)
        hypothesis_tokens = word_tokenize(hypothesis)

        # Stemming/Lemmatization
        if method == 'stem':
            stemmer = PorterStemmer()
            premise_tokens = [stemmer.stem(word) for word in premise_tokens]
            hypothesis_tokens = [stemmer.stem(word) for word in hypothesis_tokens]
        elif method == 'lemmatize':
            lemmatizer = WordNetLemmatizer()
            premise_pos_tags = pos_tag(premise_tokens)
            hypothesis_pos_tags = pos_tag(hypothesis_tokens)
            premise_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in premise_pos_tags]
            hypothesis_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in hypothesis_pos_tags]

        # POS Filtering
        if pos_filter:
            premise_tokens = filter_tokens_by_pos(premise_tokens)
            hypothesis_tokens = filter_tokens_by_pos(hypothesis_tokens)

        # Remove stop words
        if stop_w:
            premise_tokens = [word for word in premise_tokens if word not in stop_words]
            hypothesis_tokens = [word for word in hypothesis_tokens if word not in stop_words]

        # Now check token length AFTER cleaning
        if (MIN_WORD_LENGTH <= len(premise_tokens) <= MAX_WORD_LENGTH and
            MIN_WORD_LENGTH <= len(hypothesis_tokens) <= MAX_WORD_LENGTH):
            cleaned_premises.append(premise_tokens)
            cleaned_hypotheses.append(hypothesis_tokens)
            cleaned_labels.append(label)
        # else: skip row

    # Build DataFrame from all cleaned token lists
    new_dataset = pd.DataFrame({
        'premise': cleaned_premises,
        'hypothesis': cleaned_hypotheses,
        'label': cleaned_labels
    })

    return new_dataset


In [16]:
clean_train_dataset = clean_dataset(train_df, stop_w=False, method=None, pos_filter=False,MAX_WORD_LENGTH=60)
clean_test_dataset = clean_dataset(test_df,stop_w=False, method=None, pos_filter= False )
clean_validation_dataset = clean_dataset(validation_df,stop_w=False, method=None, pos_filter= False )

clean_t_dataset = clean_train_dataset['premise'] + clean_train_dataset['hypothesis']

In [17]:
clean_train_dataset.head()

,premise,hypothesis,label
0,"[pluto, rotates, once, on, its, axis, every, 6...","[earth, rotates, on, its, axis, once, times, i...",neutral
1,"[glenn, once, per, day, the, earth, rotates, a...","[earth, rotates, on, its, axis, once, times, i...",entails
2,"[geysers, periodic, gush, of, hot, water, at, ...","[the, surface, of, the, sun, is, much, hotter,...",neutral
3,"[facts, liquid, water, droplets, can, be, chan...","[evaporation, is, responsible, for, changing, ...",entails
4,"[by, comparison, the, earth, rotates, on, its,...","[earth, rotates, on, its, axis, once, times, i...",entails


## Build dictionary of words and word2id


In [18]:
print(clean_train_dataset['premise'])

0        [pluto, rotates, once, on, its, axis, every, 6...
1        [glenn, once, per, day, the, earth, rotates, a...
2        [geysers, periodic, gush, of, hot, water, at, ...
3        [facts, liquid, water, droplets, can, be, chan...
4        [by, comparison, the, earth, rotates, on, its,...
                               ...                        
22910    [which, is, not, only, the, motion, of, our, b...
22911    [the, red, star, that, celestial, curse, whose...
22912    [the, lines, in, the, spectrum, of, a, luminou...
22913    [the, radial, velocity, of, a, star, away, fro...
22914    [this, expansion, causes, the, light, from, di...
Name: premise, Length: 22915, dtype: object


In [19]:
# Create a unique word list from the cleaned dataset
unique_words = set()
for sentence in clean_t_dataset: # Both Premise and Hypothesis 
    for word in sentence:
        unique_words.add(word)

print(f"Count of unique words: {len(unique_words)}")
#No lemmatization or stemming count = 22555
#Lemmatization count = 18986
#Stemming count = 15954


Count of unique words: 22140


In [20]:
unique_words_list = sorted(list(unique_words))

In [21]:
# No. of words with each method.

word2id = {w:i for i,w in enumerate(unique_words_list)}
id2word = {i:w for i,w in enumerate(unique_words_list)}


# Build Embedding Model using Gensim and FastText


In [22]:
from gensim.models import FastText

In [23]:
# Hyperparameters
batch_size = 1024
learn_rate = 0.001
embedding_size = 200
no_of_epochs = 8
window_size = 3
vocab_size = len(unique_words)


In [24]:
fast_text_model = FastText(clean_t_dataset, # Both premise and Hypothesis 
                           vector_size=embedding_size,
                           window=window_size,
                           sg=1,
                           epochs=no_of_epochs
                           )

In [25]:
fast_text_model.wv.most_similar('saturn', topn=10)

[('saturns', 0.9542921781539917),
 ('1980', 0.8083610534667969),
 ('neptune', 0.8019237518310547),
 ('1979', 0.7835524678230286),
 ('1981', 0.7748010158538818),
 ('voyager', 0.7630559206008911),
 ('1980s', 0.7596765160560608),
 ('180', 0.7465062737464905),
 ('spacecraft', 0.744134783744812),
 ('1989', 0.7426692247390747)]

# Transformer Model Implementation


## First we need to prepare the data for to input into the transformer.


Lets add some special tokens to my vocabularies to use as separation tokens [SEP] [PAD]


In [26]:
SPECIAL_TOKENS = ['[SEP]', '[PAD]']
for tok in SPECIAL_TOKENS:
    if tok not in word2id:
        idx = len(word2id)
        word2id[tok] = idx
        id2word[idx] = tok

sep_index = word2id['[SEP]']
pad_index = word2id['[PAD]']

In [27]:
# Hyperparameters for Transformer
#max_sequence_len = 64 

In [28]:
# Build a preparation data function for the transformer
def prepare_transformer_data(df, word2id, max_sequence_len):
    input_ids = []
    labels = []
    attention_mask = []
    
    # Assuming pad_index is defined globally or passed in
    pad_index = word2id['[PAD]']
    
    for _, row in df.iterrows():
        premise = row['premise']
        hypothesis = row['hypothesis']
        label = row['label']

        tokens = premise + ['[SEP]'] + hypothesis
        
        # Truncate tokens if they are too long FIRST
        tokens = tokens[:max_sequence_len]
        
        indices = [word2id.get(w, pad_index) for w in tokens]
        
        # Create attention mask based on the TRUE length of tokens
        attn = [1] * len(indices)

        # Now, pad both indices and the attention mask
        pad_len = max_sequence_len - len(indices)
        if pad_len > 0:
            indices += [pad_index] * pad_len
            attn += [0] * pad_len # Correctly add 'pad_len' zeros

        input_ids.append(indices)
        attention_mask.append(attn)
        labels.append(0 if label == 'neutral' else 1)

    return (np.array(input_ids),
            np.array(attention_mask),
            np.array(labels))

In [29]:
x_train, train_mask, y_train = prepare_transformer_data(clean_train_dataset, word2id, 124)
x_test, test_mask, y_test = prepare_transformer_data(clean_test_dataset, word2id, 124)
x_val, val_mask, y_val  = prepare_transformer_data(clean_validation_dataset, word2id, 124)

In [30]:
x_train[0]

array([15329, 17393, 14064, 14062, 11000,  2278,  7466,   690,  6679,
        5544, 22140,  6679, 17393, 14062, 11000,  2278, 14064, 20248,
       10298, 14075,  5541, 22141, 22141, 22141, 22141, 22141, 22141,
       22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141,
       22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141,
       22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141,
       22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141,
       22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141,
       22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141,
       22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141,
       22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141,
       22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141,
       22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141, 22141,
       22141, 22141, 22141, 22141, 22141, 22141, 22141])

In [31]:
word2id['[SEP]']

22140

In [32]:
print([id2word[idx] for idx in x_train[0]])
print(y_train[0])

['pluto', 'rotates', 'once', 'on', 'its', 'axis', 'every', '639', 'earth', 'days', '[SEP]', 'earth', 'rotates', 'on', 'its', 'axis', 'once', 'times', 'in', 'one', 'day', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[

In [33]:
print(len(x_train[0]))

124


Now that we have the data as the transformer expects it, lets move to the transformer.
For the next steps i expect to build:

- Embedding Layer: Use my FastText with nn.Embeddings for the transformer
- Positional Encoding: Add the positional encoding to inject word order information.
- Transformer Encoder Layer
- Classification Head
- Forward Pass


In [66]:
#Hyperparameters Transformer 
BATCH_SIZE = 64
NUM_EPOCHS = 7
LR = 0.00008
NUM_CLASSES = 2
N_HEADS = 2
N_LAYERS = 4
DROPOUT = 0.2
WEIGHT_DECAY = 0.0005

In [67]:
import torch 
import torch.nn as nn
import math 

# Positional Encoding Class
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len):
        super().__init__()
        pe = torch.zeros(max_len, d_model) 
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0,d_model,2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe',pe)

    def forward(self,x):
        x = x + self.pe[:,:x.size(1),:]
        return x
    
# Transformer Class  
class TransformerClassifier(nn.Module):
    def __init__(self, embedding_matrix, num_classes):
        super().__init__()
        vocab_size, embed_dim = embedding_matrix.shape
        # Pre trained embedding
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix), freeze=False)
        # Train embeddings itself
        #self.embedding = nn.Embedding(vocab_size, embed_dim)

        self.pos_encoder = PositionalEncoding(embed_dim, max_len=MAX_WORD_LENGTH)

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=N_HEADS, dropout=DROPOUT) 
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=N_LAYERS) 

        self.classifier = nn.Linear(embed_dim, num_classes)

    def forward(self, x, attention_mask):
        embedded = self.embedding(x) # [batch_size, seq_len, embed_dim]
        embedded = self.pos_encoder(embedded) # Add positional encoding 

        # Create boolean padding mask
        padding_mask = (attention_mask == 0).to(x.device)

        # Transformer expects [seq_len, batch_size, embed_dim]
        encoded = self.transformer_encoder(embedded.transpose(0,1), src_key_padding_mask=padding_mask)
        # Without using the attention mask 
        #encoded = self.transformer_encoder(embedded.transpose(0,1))
        #Mean pool over seq_len: output shape [batch_size, embed_dim]
        pooled = encoded.mean(dim=0)

        logits = self.classifier(pooled)
        return logits 

Now lets build the embedding matrix from the FastText using the word2id


In [68]:
embedding_dim = fast_text_model.vector_size
vocab_size = len(word2id)

#Initialize the embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, idx in word2id.items():
    if word in fast_text_model.wv:
        embedding_matrix[idx] = fast_text_model.wv[word]
    else:
        #Initialize unkown words
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

# Convert the prepared input data 'x' to a PyTorch tensor
x_tensor = torch.LongTensor(x_train[:8]) # example mini-batch

#Instantiate your transformer Model
transformer_model = TransformerClassifier(embedding_matrix, num_classes=NUM_CLASSES)




c:\Users\DCCN9\OneDrive - UWA\Data Science\Semester July-November 2025\CITS4012 - Natural Language Processing\Team Assignment 2 NLP\.venv\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


For the last steps, lets define the optimizer and loss functin
Create the trainning loop
and build an evaluation function.


In [69]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Convert the full datasets to PyTorch tensors
# Convert the full datasets to PyTorch tensors, now including the masks
train_dataset = TensorDataset(torch.LongTensor(x_train), torch.LongTensor(train_mask), torch.LongTensor(y_train))
test_dataset = TensorDataset(torch.LongTensor(x_test), torch.LongTensor(test_mask), torch.LongTensor(y_test))
val_dataset = TensorDataset(torch.LongTensor(x_val), torch.LongTensor(val_mask), torch.LongTensor(y_val))

# The DataLoader setup doesn't need to change; it will now yield batches of three tensors.
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(transformer_model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

In [70]:
# Training function
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    # 1. Unpack all three items from the dataloader
    for inputs, masks, labels in dataloader:
        # 2. Move all tensors to the selected device
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        optimizer.zero_grad()

        # 3. Pass both inputs and masks to the model
        outputs = model(inputs, masks)
        # Without the attention mask
        #outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy


# Evaluation function
def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        # 1. Unpack all three items from the dataloader
        for inputs, masks, labels in dataloader:
            # 2. Move all tensors to the selected device
            inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)
            
            # 3. Pass both inputs and masks to the model
            outputs = model(inputs, masks)
            # Without the attention mask
            #outputs = model(inputs)

            loss = criterion(outputs, labels)

            total_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

Now we move to the trainning and evaluation


In [71]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transformer_model.to(device)
print(device)
# Main training loop
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_epoch(transformer_model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(transformer_model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} - Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

print("Training complete.")

cuda
Epoch 1/7 - Train Loss: 0.6474, Train Acc: 0.6343 - Val Loss: 0.6970, Val Acc: 0.5661
Epoch 2/7 - Train Loss: 0.5567, Train Acc: 0.7136 - Val Loss: 0.9614, Val Acc: 0.5998
Epoch 3/7 - Train Loss: 0.5159, Train Acc: 0.7461 - Val Loss: 0.7488, Val Acc: 0.6559
Epoch 4/7 - Train Loss: 0.4821, Train Acc: 0.7707 - Val Loss: 0.7780, Val Acc: 0.6544


KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = evaluate(transformer_model, test_loader, criterion, device)
print(f"FINAL TEST - Loss: {test_loss:.4f}, Acc: {test_acc:.4f}")

FINAL TEST - Loss: 0.7863, Acc: 0.7116


## Evaluation of the Model


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

transformer_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for input_ids, attention_mask, labels in test_loader:
        input_ids      = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels         = labels.to(device)

        logits = transformer_model(input_ids, attention_mask)
        preds = logits.argmax(dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

print(f"Accuracy:  {accuracy_score(all_labels, all_preds):.2f}")
print(f"F1 Score:  {f1_score(all_labels, all_preds):.2f}")
print(f"Precision: {precision_score(all_labels, all_preds):.2f}")
print(f"Recall:    {recall_score(all_labels, all_preds):.2f}")
print("-" * 40)
print(classification_report(all_labels, all_preds, target_names=["neutral", "entails"]))



print('-' * 40)
print('Hyperparameters')
print('Batch Size',BATCH_SIZE)
print('Num of Epochs',NUM_EPOCHS)
print('Learning Rate',LR)
print('Num Classes',NUM_CLASSES)
print('Number of Heads',N_HEADS)
print('Numer of Layere',N_LAYERS)
print('Embedding Dimensions',embedding_dim)
print('Dropout',DROPOUT)
print('Weight Decay',WEIGHT_DECAY)

Accuracy:  0.71
F1 Score:  0.55
Precision: 0.73
Recall:    0.43
----------------------------------------
              precision    recall  f1-score   support

     neutral       0.70      0.90      0.79      1270
     entails       0.73      0.43      0.55       842

    accuracy                           0.71      2112
   macro avg       0.72      0.66      0.67      2112
weighted avg       0.72      0.71      0.69      2112

----------------------------------------
Hyperparameters
Batch Size 64
Num of Epochs 10
Learning Rate 0.0001
Num Classes 2
Number of Heads 2
Numer of Layere 2
Embedding Dimensions 200
Dropout 0.2
Weight Decay 0.0005
